In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#transformers은 버전 3으로 고정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install konlpy

In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-ge1kqqu7
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-ge1kqqu7


In [4]:
import torch
from torch import nn
from torch import cuda
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [7]:
##GPU 사용 시 cuda:0
#device = torch.device("cuda:0")
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training koBERT Model on {device}\n{"="*44}')

Training koBERT Model on cpu


In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [9]:
import pandas as pd
import re
import csv
from sklearn.model_selection import train_test_split

In [12]:
trainset_csv = '/content/drive/MyDrive/data/topic/train_data.csv'
df = pd.read_csv(trainset_csv, index_col=0)
#df.head()

In [13]:
trainset, testset = train_test_split(df, test_size=0.1, random_state=42)

In [14]:
trainset.to_csv(path_or_buf='/content/drive/My Drive/trainset.csv',sep='\t',line_terminator='\r', encoding = 'utf-8-sig')
#trainset.head()

In [15]:
testset.to_csv(path_or_buf='/content/drive/My Drive/testset.csv',sep='\t',line_terminator='\r', encoding = 'utf-8-sig')
#testset.head()

In [16]:
#TSVDataset 함수에는 csv파일이 아닌 txt파일이 적용 가능하므로 파일 형식 변경부분
#TSVDataset에 적용할 txt파일의 경로 지정
trainset_txt = '/content/drive/My Drive/topic_train.txt'
testset_txt = '/content/drive/My Drive/topic_test.txt'
#csv파일을 txt파일로 변경
with open(trainset_txt, "w+") as my_output_file:
    with open('/content/drive/My Drive/trainset.csv', "r") as my_input_file:
        [my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()
with open(testset_txt, "w+") as my_output_file:
    with open('/content/drive/My Drive/testset.csv', "r") as my_input_file:
        [my_output_file.write(" ".join(row)+'\n') for row in csv.reader(my_input_file)]
    my_output_file.close()

In [22]:
#TSVDataset
dataset_train = nlp.data.TSVDataset(trainset_txt, field_indices=[1,2], num_discard_samples=1)
dataset_test = nlp.data.TSVDataset(testset_txt, field_indices=[1,2], num_discard_samples=1)

In [23]:
#Tokenizer
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


In [24]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [25]:
max_len = 128
batch_size = 64
warmup_ratio = 0.01
num_epochs = 2
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [26]:
data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

In [27]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

In [28]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [29]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [30]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [31]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [32]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [33]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/642 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.0315029621124268 train acc 0.09375
epoch 1 batch id 201 loss 0.5524216890335083 train acc 0.7702114427860697
epoch 1 batch id 401 loss 0.5991137027740479 train acc 0.8194747506234414
epoch 1 batch id 601 loss 0.3478442132472992 train acc 0.8386543261231281
epoch 1 train acc 0.8409511292834891


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/72 [00:00<?, ?it/s]

epoch 1 test acc 0.8800505050505051


  0%|          | 0/642 [00:00<?, ?it/s]

epoch 2 batch id 1 loss 0.20192255079746246 train acc 0.9375
epoch 2 batch id 201 loss 0.299001544713974 train acc 0.9070273631840796
epoch 2 batch id 401 loss 0.47179991006851196 train acc 0.9133416458852868
epoch 2 batch id 601 loss 0.22141699492931366 train acc 0.9168053244592346
epoch 2 train acc 0.9167883566978193


  0%|          | 0/72 [00:00<?, ?it/s]

epoch 2 test acc 0.8930713383838383


In [34]:
PATH = '/content/drive/My Drive/'
torch.save(model.state_dict(), PATH + 'model_randomstate42.pt')
#model_tc = torch.load(PATH + 'model_tc.pt')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수

##예측코드

In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [36]:
#transformers은 버전 3으로 고정
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch
!pip install konlpy
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-fo_qn9g_
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-fo_qn9g_


In [37]:
import pandas as pd
topic = pd.read_csv('/content/drive/MyDrive/data/topic/test_data.csv', encoding = 'utf-8-sig')
topic_idx = [0] * len(topic)
topic['topic_idx'] = topic_idx

In [38]:
import torch
from torch import nn
from torch import cuda
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [39]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"="*44}')

Training MNIST Model on cpu


In [40]:
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [41]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [42]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(device)

In [51]:
model.load_state_dict(torch.load("/content/drive/My Drive/model_randomstate42.pt"))

<All keys matched successfully>

In [52]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

using cached model


In [53]:
topic[["title","topic_idx"]].to_csv("/content/drive/My Drive/topic_test.txt", sep='\t', index=False)
dataset_test = nlp.data.TSVDataset("/content/drive/My Drive/topic_test.txt", field_indices=[0,1], num_discard_samples=1)
data_test = BERTDataset(dataset_test, 0, 1, tok, 128, True, False)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=64, num_workers=5)

In [54]:
def model_pred(arg_list):
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
    token_ids = token_ids.long().to(device)
    valid_length = valid_length
    segment_ids = segment_ids.long().to(device)
    label = label.long().to(device)
    pred_var = model(token_ids, valid_length, segment_ids)
    _, predict = torch.max(pred_var,1)
    arg_list.extend(predict.tolist())

In [55]:
model.eval()
list_pred = []
model_pred(list_pred)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/143 [00:00<?, ?it/s]

In [56]:
submission = pd.read_csv('/content/drive/MyDrive/data/topic/sample_submission.csv', index_col=0)

In [57]:
submission["topic_idx"] = list_pred

In [58]:
submission.to_csv('/content/drive/My Drive/koBERT_submission2.csv')